# **MIP3-Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters:

1. n = number of power units = 6
2. t = number of time periods = 15
3. DCi = Start-up costs for each unit i, where i G [1...n] --> DC1 = 10324€, DC2 = 5678€, DC3 = 7802€, DC4 = 12899€, DC5 = 4596€, DC6 = 9076€
4. SCi = Shut down costs for each unit i, where i G [1...n] --> SC1 = 2673€, SC2 = 5893€, SC3 = 982€, SC4 = 6783€, SC5 = 2596€, SC6 = 3561€
5. FCi(t) = The fixed cost for power plant i G [1...n] at time period t G [1...t]
6. VCi(t) = The variable cost for power plant i G [1...n] at time period t G [1...t]
7. LB_i = lower bound of the output power for unit i
8. UB_i = upper bound of the output power for unit i
9. INCi = Maximum power increment for power plant i
10. DECi = Maximum power decrement for power plant i
11. Demand(t) = Total power demand at time period t
12. Security = 1.1 (10% higher than the demand)

Decision Variables:

1. Xi(t) = 1, if power unit i is ON at time period t, 0, otherwise
2. Yi(t) = 1, if power unit i is ON at time period t and was OFF at ti

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Sure, here is the Pyomo code for the given optimization problem. 

```python
from pyomo.environ import *

#number of power units and time periods
n = 6 
t = 15

#creating the model
model = ConcreteModel()

index_i = range(n)
index_t = range(t)

#parameters
DCi = [10324, 5678, 7802, 12899, 4596, 9076]  #assuming
SCi = [2673, 5893, 982, 6783, 2596, 3561]  #assuming
FCi = [[0 for _t in index_t] for _i in index_i]  #assuming
VCi = [[0 for _t in index_t] for _i in index_i]  #assuming
LB_i = [0 for _i in index_i]  #assuming
UB_i = [0 for _i in index_i]  #assuming
INCi = [0 for _i in index_i]  #assuming
DECi = [0 for _i in index_i]  #assuming
Demand = [0 for _t in index_t]  #assuming
Security = 1.1 

#decision variables
model.Xi = Var(index_i, index_t, within=Binary)
model.Yi = Var(index_i, index_t, within=Binary)
model.Zi = Var(index_i, index_t, within=Binary)
model.Pi = Var(index_i, index_t, within=Reals)

#Objective function
def objective_rule(model):
    return sum(sum(DCi[i]*model.Yi[i,t] + SCi[i]*model.Zi[i,t] + FCi[i][t]*model.Xi[i,t] + VCi[i][t]*model.Pi[i,t] for i in index_i) for t in index_t)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Constraints
model.constraints = ConstraintList()

#Security Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Security*Demand[_t])
   
#Power Demand Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Demand[_t])
    
#Minimum & Maximum Power Output
for i in index_i:
    for _t in index_t:
        model.constraints.add(LB_i[i]*model.Xi[i,_t] <= model.Pi[i,_t])
        model.constraints.add(model.Pi[i,_t] <= UB_i[i]*model.Xi[i,_t])

#Increment Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i, _t-1] <= INCi[i]*model.Xi[i,_t])

#Decrement Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i,_t-1] >= -DECi[i]*model.Xi[i,_t])

#Start-up Indicator
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Yi[i,_t] >= model.Xi[i,_t] - model.Xi[i,_t-1])

#Shut-down Indicator:
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Zi[i,_t] >= model.Xi[i,_t-1] - model.Xi[i,_t])

#Initial condition constraints
for i in index_i:
    model.constraints.add(model.Yi[i,0] == model.Xi[i,0])
    model.constraints.add(model.Zi[i,0] == 0)

#Solver
solver = SolverFactory('glpk')
solver.solve(model)
```

In the code above, I filled the parameters with dummy values for the illustrative purposes. Please replace them with your actual data. The rest of the code will remain the same.

### **Run the code Generated by GPT4**

In [9]:
from pyomo.environ import *

# number of power units and time periods
n = 6
t = 15

# creating the model
model = ConcreteModel()

index_i = range(n)
index_t = range(t)

# parameters
DCi = [10324, 5678, 7802, 12899, 4596, 9076]  #assuming
SCi = [2673, 5893, 982, 6783, 2596, 3561]  #assuming
FCi = [[0 for _t in index_t] for _i in index_i]  #assuming
VCi = [[0 for _t in index_t] for _i in index_i]  #assuming
LB_i = [0 for _i in index_i]  #assuming
UB_i = [0 for _i in index_i]  #assuming
INCi = [0 for _i in index_i]  #assuming
DECi = [0 for _i in index_i]  #assuming
Demand = [0 for _t in index_t]  #assuming
Security = 1.1

# decision variables
model.Xi = Var(index_i, index_t, within=Binary)
model.Yi = Var(index_i, index_t, within=Binary)
model.Zi = Var(index_i, index_t, within=Binary)
model.Pi = Var(index_i, index_t, within=Reals)

# Objective function
def objective_rule(model):
    return sum(sum(DCi[i]*model.Yi[i,t] + SCi[i]*model.Zi[i,t] + FCi[i][t]*model.Xi[i,t] + VCi[i][t]*model.Pi[i,t] for i in index_i) for t in index_t)
model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraints
model.constraints = ConstraintList()

# Security Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Security*Demand[_t])

# Power Demand Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Demand[_t])

# Minimum & Maximum Power Output
for i in index_i:
    for _t in index_t:
        model.constraints.add(LB_i[i]*model.Xi[i,_t] <= model.Pi[i,_t])
        model.constraints.add(model.Pi[i,_t] <= UB_i[i]*model.Xi[i,_t])

# Increment Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i, _t-1] <= INCi[i]*model.Xi[i,_t])

# Decrement Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i,_t-1] >= -DECi[i]*model.Xi[i,_t])

# Start-up Indicator
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Yi[i,_t] >= model.Xi[i,_t] - model.Xi[i,_t-1])

# Shut-down Indicator:
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Zi[i,_t] >= model.Xi[i,_t-1] - model.Xi[i,_t])

# Initial condition constraints
for i in index_i:
    model.constraints.add(model.Yi[i,0] == model.Xi[i,0])
    model.constraints.add(model.Zi[i,0] == 0)

# Solver
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 558, 'Number of variables': 360, 'Number of nonzeros': 1218, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.010285377502441406}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [13]:
from pyomo.environ import *

# number of power units and time periods
n = 6
t = 15

# creating the model
model = ConcreteModel()

index_i = range(n)
index_t = range(t)

# data inputted by human
DCi = [10324,5678,7802, 12899,4596,9076]
SCi = [2673, 5893, 982, 6783, 2596, 3561]
FCi = [2000, 3000, 2500, 4000, 3500, 4500]
VCi = [[20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
      [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
      [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
      [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
      ]
LB_i = [50, 40, 30, 60, 55,65]
UB_i = [500, 600, 550, 700, 650, 750]
INCi = [100, 120, 110, 130, 125, 140]
DECi = [90, 110, 100, 120, 115, 130]
Demand = [1000,1200, 1300, 1100, 1500, 1400, 1600, 1300, 1700, 1800, 1900, 1600, 2000, 1800, 1700]
#end
Security = 1.1

# decision variables
model.Xi = Var(index_i, index_t, within=Binary)
model.Yi = Var(index_i, index_t, within=Binary)
model.Zi = Var(index_i, index_t, within=Binary)
model.Pi = Var(index_i, index_t, within=Reals)

# Objective function
def objective_rule(model):
    return sum(sum(DCi[i]*model.Yi[i,t] + SCi[i]*model.Zi[i,t] + FCi[i]*model.Xi[i,t] + VCi[i][t]*model.Pi[i,t] for i in index_i) for t in index_t)
model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraints
model.constraints = ConstraintList()

# Security Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Security*Demand[_t])

# Power Demand Constraint
for _t in index_t:
    model.constraints.add(sum(model.Pi[i,_t] for i in index_i) >= Demand[_t])

# Minimum & Maximum Power Output
for i in index_i:
    for _t in index_t:
        model.constraints.add(LB_i[i]*model.Xi[i,_t] <= model.Pi[i,_t])
        model.constraints.add(model.Pi[i,_t] <= UB_i[i]*model.Xi[i,_t])

# Increment Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i, _t-1] <= INCi[i]*model.Xi[i,_t])

# Decrement Constraint
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Pi[i,_t] - model.Pi[i,_t-1] >= -DECi[i]*model.Xi[i,_t])

# Start-up Indicator
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Yi[i,_t] >= model.Xi[i,_t] - model.Xi[i,_t-1])

# Shut-down Indicator:
for i in index_i:
    for _t in range(1, t):
        model.constraints.add(model.Zi[i,_t] >= model.Xi[i,_t-1] - model.Xi[i,_t])

# Initial condition constraints
for i in index_i:
    model.constraints.add(model.Yi[i,0] == model.Xi[i,0])
    model.constraints.add(model.Zi[i,0] == 0)

# Solver
solver = SolverFactory('glpk')
solver.solve(model)
print(model.objective())

825970.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**